In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs, make_moons

In [64]:
#importo datasets de test y train

df_test=pd.read_parquet('test.parquet')
df_train= pd.read_parquet('train.parquet')

# elimino columnas que no me son utiles
df_test=df_test.drop(['url','region_url','cats_allowed','dogs_allowed','image_url','smoking_allowed'],axis=1)
df_train=df_train.drop(['url','region_url','cats_allowed','dogs_allowed','image_url','smoking_allowed'],axis=1)

cree funciones para eliminar outliers

In [65]:
#funcion para tratamiento de outliers dando de argumento para limites superior e inferior manualmente
def no_outliers(df,columna,upper,lower):
    #deficicion de upper limit en categoria precio
    upper_limit = upper

    #deficicion de lower limit en categoria precio
    lower_limit = lower

    #creacion de nueva df[columna] el con outliers remplazados por uper y lowe limit respectivamente
    df[columna] = np.where(df[columna] > upper_limit,upper_limit,np.where(df[columna] < lower_limit,lower_limit,df[columna]))
    return 

#funcion para tratamiento de outliers dando argumentos para limites usando std
def no_outliers_std(df,columna,std_no):
    #deficicion de upper limit en categoria precio
    upper_limit = df[columna].mean() + std_no*df[columna].std()

    #deficicion de lower limit en categoria precio
    lower_limit = df[columna].mean() - std_no*df[columna].std()

    #creacion de nueva df[columna] el con outliers remplazados por uper y lowe limit respectivamente
    df[columna] = np.where(df[columna] > upper_limit,upper_limit,np.where(df[columna] < lower_limit,lower_limit,df[columna]))
    return 

normalizamos la data de df_train

In [66]:

#creacion de dummies en columna type
hy=pd.get_dummies(df_train['type'])
#elimino columna type para ser remplazada con dummies
df_train.drop('type',inplace= True, axis=1)
#incorporo columnas dummies en datasets
df_train=pd.concat([df_train,hy],axis=1)

#creacion de dataframe para uso en entrenamiento 

df_tr_supervisado=df_train[['id','price','sqfeet','beds','baths','wheelchair_access','electric_vehicle_charge','comes_furnished','apartment', 'condo', 'cottage/cabin', 'duplex',
       'flat', 'house', 'in-law', 'loft', 'manufactured', 'townhouse']]
#elimine las filas con precio inferior a 200 y superior a 5000 en mi df de entrenamiento 
df_tr_supervisado= df_tr_supervisado[df_tr_supervisado['price'] < 5000]
df_tr_supervisado= df_tr_supervisado[df_tr_supervisado['price'] > 200]

#lista de columnas a procesar con funcion de tratamiento de outliers, en esta no incluy el precio ya que voy a cambiar el margen de outlier

lis_col_usadas=['sqfeet','wheelchair_access','electric_vehicle_charge','comes_furnished']

#for loop para implementar funcin de outlier usando lista descrita antes
for element in lis_col_usadas:
    no_outliers_std(df_tr_supervisado, element,3)

#implementacion de funcion en columna baths, en esta elegi los limites superior e inferior manualmente, basandome en la datos que visualice utilizando, value_count en la columna 
#deficicion de upper y lowerlimit en categoria bath
no_outliers(df_tr_supervisado,'baths',6.5,0.5)

#deficicion de upper y lowerlimit en categoria beds
no_outliers(df_tr_supervisado,'beds',8,1)


#implementacion de funcion en columna precio
#no_outliers_std(df_tr_supervisado,'price',1)

#creacion de columna de precio low usando el siguiente criterio (propiedades con precio menor a 999 low 1)(propiedades con precio mayor a 999 NO low 0)
df_tr_supervisado['catg_price'] = np.where(df_tr_supervisado['price']> 999, 0, 1)
#realize un print con el conteo de valores en la columna de categoria para comprobar que este balanceado 
df_tr_supervisado['catg_price'].value_counts()


0    184925
1    158290
Name: catg_price, dtype: int64

normalizamos data de df_test

In [67]:
#creacion de dummies en columna type
hy=pd.get_dummies(df_test['type'])
#elimino columna type para ser remplazada con dummies
df_test.drop('type',inplace= True, axis=1)
#incorporo columnas dummies en datasets
df_test=pd.concat([df_test,hy],axis=1)

#lista de columnas a procesar con funcion de tratamiento de outliers, en esta no incluy el precio ya que voy a cambiar el margen de outlier

lis_col_usadas=['sqfeet','wheelchair_access','electric_vehicle_charge','comes_furnished']

#for loop para implementar funcin de outlier usando lista descrita antes
for element in lis_col_usadas:
    no_outliers_std(df_test, element,3)

#implementacion de funcion en columna baths, en esta elegi los limites superior e inferior manualmente, basandome en la datos que visualice utilizando, value_count en la columna 
#deficicion de upper y lowerlimit en categoria bath
no_outliers(df_test,'baths',6.5,0.5)

#deficicion de upper y lowerlimit en categoria beds
no_outliers(df_test,'beds',8,1)

In [68]:
#creacion de dataset para uso en entrenamiento y graficas de correlacion, en este exluyo los datos de id y de precio
dftrs=df_tr_supervisado[['catg_price','sqfeet','beds','baths','wheelchair_access','electric_vehicle_charge','comes_furnished','apartment', 'condo', 'cottage/cabin', 'duplex',
       'flat', 'house', 'in-law', 'loft', 'manufactured', 'townhouse']]

en esta seccion creo las variables e implemento lo necesario para entrenar el modelo 

In [69]:
# Denotamos X con las variables explicativas que vamos a usar
X = dftrs[['sqfeet','beds','baths','wheelchair_access','electric_vehicle_charge','comes_furnished','apartment', 'condo', 'cottage/cabin', 'duplex',
       'flat', 'house', 'in-law', 'loft', 'manufactured', 'townhouse']]
# Denotamos y con la variable objetivo
y = dftrs['catg_price'] 


#Separamos los datos de entrenamiento de los datos de prueba
from sklearn.model_selection import train_test_split

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, y, test_size = 0.25, random_state=0)

#Estandarizamos (obviar esta parte por el momento, simplemente ejecutar)
from sklearn.preprocessing import StandardScaler

#sc_X = StandardScaler()
#X_Train = sc_X.fit_transform(X_Train)
#X_Test = sc_X.transform(X_Test)

#importamos el modelo de arboles de decicion
from sklearn.tree import DecisionTreeClassifier

# Instanciamos un objeto de la clase DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth = 500, random_state = 40) 

# Entrenamos el modelo
clf.fit(X_Train, Y_Train)

DecisionTreeClassifier(max_depth=500, random_state=40)

exploreo caracteristicas de mi modelo 

In [70]:
# Exploramos algunas de las características del modelo entrenado. Para pensar, 
# ¿qúe representa cada uno de estos valores?

print(clf.classes_)
print(clf.n_classes_)
print(clf.max_features_)
print(clf.feature_importances_)



[0 1]
2
16
[6.94103771e-01 6.69147297e-02 8.60618449e-02 3.22481022e-02
 1.59602332e-02 2.90764794e-02 1.53558812e-02 1.19236264e-02
 1.77741207e-03 5.88277295e-03 1.04359069e-03 1.54349926e-02
 6.64909960e-04 1.53093029e-03 1.29734701e-02 9.04725287e-03]


In [71]:
#creo nueva variable de prediccion usando variable que contiene mis datos de test

y_pred = clf.predict(X_Test)

#importar funcion accuracy y print el acuracy_score
from sklearn.metrics import accuracy_score
#imprimo el acurracy score
print(accuracy_score(Y_Test,y_pred))

0.7484616101813435


In [72]:
#compruebo el acurracy en mis con mis datos de train 
y_pred = clf.predict(X_Train)

#importar funcion accuracy y print el acuracy_score
from sklearn.metrics import accuracy_score

print(accuracy_score(Y_Train,y_pred))

0.7797568868463275


implementacion y uso de modelo ya entrenado en datos de test y exporto resultados en csv

In [73]:
#creo dataframe con las mismas columnas que mi df usado en el entrenamiento 

test = df_test[['sqfeet','beds','baths','wheelchair_access','electric_vehicle_charge','comes_furnished','apartment', 'condo', 'cottage/cabin', 'duplex',
       'flat', 'house', 'in-law', 'loft', 'manufactured', 'townhouse']]

#Ejecuto la prediccion en mi DF test
prediccion_data_test = clf.predict(test)

#creacion de df vacio para agregar datos de prediccion
dfpred=pd.DataFrame()

#agregue datos de prediccion
dfpred['pred']=prediccion_data_test

#Exporar resultados de prediccion en formato csv 
dfpred.to_csv(r'ALEX-MGS.csv', index=False)

In [249]:
#df_tr_supervisado['catg_price'] = np.where(df_tr_supervisado['price']>1999,2,np.where(df_tr_supervisado['price'] < 1000,0,1))
#df_tr_supervisado['catg_price'] = np.where(df_tr_supervisado['price']> 1000,1,0)
#df_tr_supervisado['catg_price'].value_counts()

1    182051
0    161164
Name: catg_price, dtype: int64

unique_region
unique_type
